In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# Örnek 5.6: camShift Algoritması

video = cv2.VideoCapture("0")
ret, prevFrame = video.read()

faceDetection = cv2.CascadeClassifier('DATA/haarcascades/haarcascade_frontalface_default.xml')
faceLocation = faceDetection.detectMultiScale(prevFrame)

(faceX, faceY, w, h) = tuple(faceLocation[0])
faceTracking = (faceX, faceY, w, h)

roi = prevFrame[faceY:faceY + h, faceX:faceX + w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

hist_roi = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
cv2.normalize(hist_roi, hist_roi, 0, 255, cv2.NORM_MINMAX)

paramaters = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, nextFrame = video.read()
    hsv_NextFrame = cv2.cvtColor(nextFrame, cv2.COLOR_BGR2HSV)

    target = cv2.calcBackProject([hsv_NextFrame], [0], hist_roi, [0, 180], 1)

    ret, faceTracking = cv2.CamShift(target, faceTracking, paramaters)
    
    points = cv2.boxPoints(ret)
    points = np.int0(points)

    mark = cv2.polylines(nextFrame, [points], True, (0, 255, 255), 5)

    cv2.imshow('Takip', mark)

    k = cv2.waitKey(10) & 0xFF

    if k == ord('q'):
        break

video.release()
cv2.destroyAllWindows()